torchtext==0.6 사용

In [29]:
from torchtext.data import Field, TabularDataset, BucketIterator
import torch
from pathlib import Path
import pandas as pd

In [30]:
dataset_dir = Path('./data')
train_data_file = 'train_data.csv'
valid_data_file = 'valid_data.csv'
test_data_file = 'test_data.csv'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [19]:
pd.read_csv(dataset_dir/train_data_file).head()

,Sentence,Label
0,I need to see Gina.,1
1,"Well, she’s actually not here.",1
2,"Well I know she is, cos I saw her come in and ...",0


tokenize 정의

In [3]:
simple_tokenize = lambda x:x.split()

Field - 이루어져야 할 전처리를 지정한다

In [5]:
TEXT = Field( #Sentence 열에 적용할 Field
    sequential = True, #문장이 들어오면 True, 한 단어면 False
    use_vocab = True, #단어장을 만들 것이면 True
    tokenize = simple_tokenize, #tokenize 전달
    lower = True, #모두 소문자로
    batch_first = True #배치 차원이 먼저오도록
)


LABEL = Field( #Label 열에 적용할 Field
    sequential = False, #문장이 아니므로
    use_vocab = False, #단어장 필요 없음
    batch_first = True #배치 차원이 먼저오도록
)

fields 딕셔너리로 묵는다

In [6]:
#방법 1
fileds = {
    'Sentence' : ('text',TEXT), #원래 데이터파일에 저장된 열이름 : (내가 사용할 키, Field)
    'Label' : ('label', LABEL)
    }

#방법 2
fields = [('text', TEXT), ('label',LABEL)] #원래 데이터파일에 저장된 순서대로 대응되는 튜플 나열

TabularDataset - 데이터를 불러온다

In [22]:
train_data, valid_data, test_data =\
    TabularDataset.splits(
        path = dataset_dir,
        train = train_data_file,
        validation = valid_data_file,
        test = test_data_file,
        format = 'csv',
        fields = fields, #만들어둔 fields를 전달
        skip_header = 1 #첫번째 행에는 column명이 들어가므로 skip
    )

In [26]:
train_data[0].__dict__

{'text': ['i', 'need', 'to', 'see', 'gina.'], 'label': '1'}

vocab만들기 - 단어를 숫자화하는 표

In [27]:
TEXT.build_vocab(train_data, min_freq = 2) #2번이상 나온 단어만 사용

BuckeIterator - iterator만들기(Data Loader)

In [39]:
train_iterator, valid_iterator, test_iterator =\
    BucketIterator.splits(
        (train_data, valid_data, test_data),
        batch_size = 2,
        device = device,
        sort = False #정렬 하지 않을 것
    )

In [54]:
for batch in train_iterator:
    print(batch.text)
    print(batch.label)
    break

tensor([[0, 2, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([0, 1])
